In [ ]:
############################################################
# Author: Babatunde Kalejaiye
# Student ID: KAL20504783
# Date: 10/10/2021
# Course-Work 3: Cancer Treatment Decision Simulator
############################################################


import pygame
import sys
import numpy as np
import random
import copy

# Constants variables
SCREEN_WIDTH = 700
SCREEN_HEIGHT = 700
BG_COLOR = (0, 0, 139)
LINE_COLOR = (23, 160, 135)
CROSS_COLOR = (66, 66, 66)
LINE_WIDTH = 10
CIRCLE_WIDTH = 10
CROSS_WIDTH = 15
ROWS = 3
COLS = 3
SQSIZE = SCREEN_WIDTH // COLS
CIRCLE_COLOR = (239, 231, 200)
RADIUS = SQSIZE // 6
OFFSET = 80

# Initialize the game with error handling
try:
    pygame.init()
    screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
    pygame.display.set_caption('Cancer Treatment Decision Simulator')
except pygame.error as e:
    print("Failed to initialize Pygame:", e)
    sys.exit(1)

try:
    screen.fill(BG_COLOR)
except pygame.error as e:
    print("Failed to fill screen with background color:", e)
    sys.exit(1)


# Classes to draw the board, make moves and decide the outcome
class Board:
    def __init__(self):
        self.squares = np.zeros((ROWS, COLS))
        self.marked_squares = 0
    
    # Function to draw the final state of the game
    def final_state(self, show=False):
        for col in range(COLS):
            if self.squares[0][col] == self.squares[1][col] == self.squares[2][col] != 0:
                if show:
                    color = CIRCLE_COLOR if self.squares[0][col] == 2 else CROSS_COLOR
                    iPos = (col * SQSIZE + SQSIZE // 2, 20)
                    fPos = (col * SQSIZE + SQSIZE // 2, SCREEN_HEIGHT - 20)
                    pygame.draw.line(screen, color, iPos, fPos, LINE_WIDTH)
                return self.squares[0][col]

        for row in range(ROWS):
            if self.squares[row][0] == self.squares[row][1] == self.squares[row][2] != 0:
                if show:
                    color = CIRCLE_COLOR if self.squares[row][0] == 2 else CROSS_COLOR
                    iPos = (20, row * SQSIZE + SQSIZE // 2)
                    fPos = (SCREEN_WIDTH - 20, row * SQSIZE + SQSIZE // 2)
                    pygame.draw.line(screen, color, iPos, fPos, CROSS_WIDTH)
                return self.squares[row][0]

        if self.squares[0][0] == self.squares[1][1] == self.squares[2][2] != 0:
            if show:
                color = CIRCLE_COLOR if self.squares[1][1] == 2 else CROSS_COLOR
                iPos = (20, 20)
                fPos = (SCREEN_WIDTH - 20, SCREEN_HEIGHT - 20)
                pygame.draw.line(screen, color, iPos, fPos, CROSS_WIDTH)
            return self.squares[1][1]

        if self.squares[2][0] == self.squares[1][1] == self.squares[0][2] != 0:
            if show:
                color = CIRCLE_COLOR if self.squares[1][1] == 2 else CROSS_COLOR
                iPos = (20, SCREEN_HEIGHT - 20)
                fPos = (SCREEN_WIDTH - 20, 20)
                pygame.draw.line(screen, color, iPos, fPos, CROSS_WIDTH)
            return self.squares[1][1]

        return 0

    # Function to mark the square with the player's move
    def mark_square(self, row, col, player):
        self.squares[row][col] = player
        self.marked_squares += 1
    
    # Function to check if the square is empty
    def empty_square(self, row, col):
        try:
            return self.squares[row][col] == 0
        except IndexError:
            print("Invalid square coordinates:", row, col)
            return False
    
    # Function to get the empty squares
    def get_empty_squares(self):
        return [(i, j) for i in range(3) for j in range(3) if self.squares[i][j] == 0]

    # Function to check if the board is full
    def is_full(self):
        return self.marked_squares == 9
    
    # Function to check if the board is empty
    def is_empty(self):
        return self.marked_squares == 0

# Class to decide the AI's move
class AI:
    def __init__(self, level=1, player=2):
        self.level = level
        self.player = player

    # Function to decide the AI's move
    def random_move(self, board):
        empty_squares = board.get_empty_squares()
        if empty_squares:
            index = random.randrange(0, len(empty_squares))
            return empty_squares[index]
        else:
            return None

    # Function to decide the AI's move using the minimax algorithm
    def minimax(self, board, is_maximizing):
        case = board.final_state()
        if case == 1:
            return 1, None
        elif case == 2:
            return -1, None
        elif board.is_full():
            return 0, None

        if is_maximizing:
            max_score = -1000
            best_move = None
            empty_sqrs = board.get_empty_squares()
            for row, col in empty_sqrs:
                temp_board = copy.deepcopy(board)
                temp_board.mark_square(row, col, 1)
                eval = self.minimax(temp_board, False)[0]
                if eval > max_score:
                    max_score = eval
                    best_move = (row, col)
            return max_score, best_move

        elif not is_maximizing:
            min_score = 1000
            best_move = None
            empty_sqrs = board.get_empty_squares()
            for row, col in empty_sqrs:
                temp_board = copy.deepcopy(board)
                temp_board.mark_square(row, col, self.player)
                eval = self.minimax(temp_board, True)[0]
                if eval < min_score:
                    min_score = eval
                    best_move = (row, col)
            return min_score, best_move
    
    # Function to evaluate the AI's move
    def evaluate(self, main_board):
        if self.level == 0:
            move = self.random_move(main_board)
        else:
            evaluate, move = self.minimax(main_board, False)
        print(f"Cancer has chosen to mark the square in pos {move} with the evaluation of {evaluate}")
        return move

# Class to manage the game
class Game:
    def __init__(self):
        self.board = Board()
        self.ai = AI()
        self.player = 1
        self.game_mode = 'ai'
        self.running = True
        self.draw_lines()
        self._has_winner = False
        self.patient_profile = {}

    # Function to make a move
    def make_move(self, row, col):
        self.update_patient_profile(row, col)
        self.board.mark_square(row, col, self.player)
        self.draw_fig(row, col)
        self.next_turn()
        self.process_move()

    # Function to update the patient's profile
    def update_patient_profile(self, row, col):
        self.patient_profile[(row, col)] = 'Treatment Decision'

    # Function to process the move
    def process_move(self):
        if self.has_winner():
            self.running = False
            print(f"Cancer Treatment has been decided for patient {self.player}!")
            self.display_patient_profile()
            self.display_outcome_message()
        elif self.is_tied():
            self.running = False
            print(f"Patient {self.player} has been diagnosed with cancer!")
            self.display_patient_profile()
            self.display_outcome_message()
    
    # Function to display the patient's profile
    def display_patient_profile(self):
        print("Patient Profile:")
        for move, decision in self.patient_profile.items():
            print(f"Move {move}: {decision}")

    # Function to display the outcome message
    def display_outcome_message(self):
        if self.has_winner():
            print("The patient's cancer treatment was successful!")
        else:
            print("The patient's cancer treatment was not successful.")

    # Function to check if there is a winner
    def has_winner(self):
        return self.board.final_state(show=False) != 0

    
    # Function to check if the game is tied
    def is_tied(self):
        return self.board.is_full() and not self.has_winner()

    # Function to draw the lines
    def draw_lines(self):
        try:
            screen.fill(BG_COLOR)
            for i in range(ROWS):
                pygame.draw.line(screen, LINE_COLOR, (0, i * SCREEN_HEIGHT / ROWS),
                                 (SCREEN_WIDTH, i * SCREEN_HEIGHT / ROWS), LINE_WIDTH)
            for i in range(COLS):
                pygame.draw.line(screen, LINE_COLOR, (i * SCREEN_WIDTH / COLS, 0),
                                 (i * SCREEN_WIDTH / COLS, SCREEN_HEIGHT), LINE_WIDTH)
            pygame.display.update()
        except pygame.error as e:
            print("Failed to draw lines:", e)
            sys.exit(1)

    # Function to draw the figure
    def draw_fig(self, row, col):
        try:
            if self.player == 1:
                start_desc = (col * SQSIZE + OFFSET, row * SQSIZE + OFFSET)
                end_desc = (col * SQSIZE + SQSIZE - OFFSET, row * SQSIZE + SQSIZE - OFFSET)
                pygame.draw.line(screen, CROSS_COLOR, start_desc, end_desc, CROSS_WIDTH)
                start_asc = (col * SQSIZE + OFFSET, row * SQSIZE + SQSIZE - OFFSET)
                end_asc = (col * SQSIZE + SQSIZE - OFFSET, row * SQSIZE + OFFSET)
                pygame.draw.line(screen, CROSS_COLOR, start_asc, end_asc, CROSS_WIDTH)
            elif self.player == 2:
                center = (col * SCREEN_WIDTH / COLS + SCREEN_WIDTH / COLS // 2,
                          row * SCREEN_HEIGHT / ROWS + SCREEN_HEIGHT / ROWS // 2)
                pygame.draw.circle(screen, CIRCLE_COLOR, center, RADIUS, CIRCLE_WIDTH)
        except pygame.error as e:
            print("Failed to draw figure:", e)
            sys.exit(1)


    # Function to change the turn
    def next_turn(self):
        self.player = self.player % 2 + 1


    # Function to change the game mode
    def change_game_mode(self):
        self.game_mode = 'ai' if self.game_mode == 'playerVSplayer' else 'playerVSplayer'
        print(f"Game mode changed to {self.game_mode}")
        
    def print_welcome_message(self):
        print("\033[1m\033[95mWelcome to the Treatment Decision Simulator!\033[0m")
        print("\033[93mPLEASE READ THE FOLLOWING INSTRUCTIONS BEFORE STARTING THE GAME\033[0m")
        print("\033[96mIn this game, your moves represent treatment decisions\033[0m")
        print("\033[96mand the AI opponent (cancer) reacts accordingly.\033[0m")
        print("\033[96mThe patient profile will evolve based on your decisions\033[0m")
        print("\033[96mand the challenges posed by cancer.\033[0m")
        print("\033[96mThe game ends when the patient's outcome is determined.\033[0m")
        print("\033[92mWhen you are ready to start, click on the desired cell to begin the game.\033[0m")
        print("\033[92mif no winner, press '\033[97mr\033[92m' to reset the game.\033[0m")
        print("\033[92mGame mode can also be changed to Doctor vs Doctor by clicking '\033[97mg\033[92m'.\033[0m")
        print("\033[92mPress '\033[97m0\033[92m' to change AI's level to random.\033[0m")
    
    # Function to make a move
    def isOver(self):
        return self.board.final_state(show=True) != 0 or self.board.is_full()

    # Function to reset the game
    def reset(self):
        self.__init__()



# Main function to run the game
def main():
    
    game = Game()
    game.print_welcome_message()  # Call the method to print the welcome message
    
    try:
        game = Game()
        board = game.board
        ai = game.ai
        # Main game loop
        while True:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()
                    
                # Handle the user's input
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_g:
                        game.change_game_mode()
                    if event.key == pygame.K_r:
                        game.reset()
                        board = game.board
                        ai = game.ai
                    if event.key == pygame.K_0:
                        ai.level = 0
                # Handle the user's click
                if event.type == pygame.MOUSEBUTTONDOWN:
                    pos = event.pos
                    row = pos[1] // SQSIZE
                    col = pos[0] // SQSIZE
                    if board.empty_square(row, col) and game.running:
                        game.make_move(row, col)
                        if game.isOver():
                            game.running = False
            # Handle the AI's move
            if game.game_mode == 'ai' and game.player == ai.player and game.running:
                pygame.display.update()
                row, col = ai.evaluate(board)
                game.make_move(row, col)
                if game.isOver():
                    game.running = False
            # Update the display
            pygame.display.update()
    except Exception as e:
        print("An error occurred:", e)


main()
